# An Example 2D optimisation
## Inspired by the following notebook:
[this blog post](https://thuijskens.github.io/2016/12/29/bayesian-optimisation/) referring to 
[this notebook](https://github.com/thuijskens/bayesian-optimization/blob/master/ipython-notebooks/svm-optimization.ipynb)

target function and inspiration for plotting from [here](https://github.com/fmfn/BayesianOptimization/issues/18)

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%matplotlib inline
#%config InlineBackend.figure_format = 'svg'
import matplotlib.pyplot as plt
import seaborn as sns; sns.set() # prettify matplotlib

import numpy as np
import sklearn.gaussian_process as gp

In [ ]:
# local modules
import turbo as tb
import turbo.modules as tm
import turbo.plotting as tp
import turbo.gui as tg

In [ ]:
# Make deterministic
np.random.seed(0)

# Target Function

In [ ]:
xmin, xmax = 0, 6
ymin, ymax = 0, 6
x = np.linspace(xmin, xmax, 100)
y = np.linspace(ymin, ymax, 100)

X, Y = np.meshgrid(x, y)

# vectorize needed for accessing noise
#@np.vectorize
def f(x, y):
    ''' from https://github.com/fmfn/BayesianOptimization/issues/18 '''
    a = np.exp(-( (x - 2)**2/0.7 + (y - 4)**2/1.2) + (x - 2)*(y - 4)/1.6 )
    b = np.exp(-( (x - 4)**2/3 + (y - 2)**2/2.) )
    c = np.exp(-( (x - 4)**2/0.5 + (y - 4)**2/0.5) + (x - 4)*(y - 4)/0.5 )
    d = np.sin(3.1415 * x)
    e = np.exp(-( (x - 5.5)**2/0.5 + (y - 5.5)**2/.5) )
    return 2*a + b - c + 0.17 * d + 2*e

Z = f(X, Y)

tp.surface_3D(X, Y, Z)

def calc_best():
    # 2.06434770773
    x = np.linspace(2.0, 2.2, 1000)
    y = np.linspace(3.9, 4.1, 1000)
    return np.max(f(*np.meshgrid(x, y)))
best_z = calc_best()

# Helper Functions

In [ ]:
bounds = [
    ('x', xmin, xmax),
    ('y', ymin, ymax)
]

op = tb.Optimiser(f, 'max', bounds, pre_phase_trials=4, settings_preset='default')
op.surrogate = tm.SciKitGPSurrogate(model_params=dict(
    alpha = 1e-3, # larger => more noise. Default = 1e-10
    kernel = 1.0 * gp.kernels.RBF(length_scale_bounds=(1e-2, 5)),
    n_restarts_optimizer = 10,
    normalize_y = True
))
op.acq_func_factory = tm.UCB.Factory(beta=1)

rec = tp.PlottingRecorder(op)

In [ ]:
tg.OptimiserProgressBar(op)
op.run(max_trials=15)

In [ ]:
op.get_incumbent()

In [ ]:
tp.plot_error(rec, true_best=best_z);

In [ ]:
tp.plot_surrogate_hyper_params_2D(rec);

In [ ]:
tp.interactive_plot_trial_1D(rec)

In [ ]:
tp.interactive_plot_trial_2D(rec, true_objective=f);

# Try optimising the same function with random search

In [ ]:
ro = tb.Optimiser(f, 'max', bounds=bounds, pre_phase_trials=float('inf'), settings_preset='random_search')
rrec = tp.PlottingRecorder(ro)

In [ ]:
tg.OptimiserProgressBar(ro)
ro.run(max_trials=100)

In [ ]:
ro.get_incumbent()

In [ ]:
tp.plot_error(rrec, true_best=best_z, log_scale=False, fig_ax=plt.subplots(figsize=(8, 3)));

In [ ]:
tp.interactive_plot_trial_1D(rrec, param='x');

In [ ]:
tp.interactive_plot_trial_2D(rrec, x_param='x', y_param='y', true_objective=f);